**Vanilla ResNet code: [Keras Doc Example][1] <br/>
Growing Sets method implemented by Ömer Kurttekin & Serkan K. Bahşi, April 2020 <br/>
NOVA Research Lab**<br/>


  [1]: https://keras.io/examples/cifar10_resnet/

In [0]:
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import model_from_json
#from tensorflow import keras
from random import shuffle
import numpy as np
import os
import h5py
import json

import datetime
#from google.colab import drive
#drive.mount('/content/drive')

# Training parameters
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = False # data_augmentation=True is not tested for this notebook.
num_classes = 100

In [0]:
subtract_pixel_mean = True
n = 3
# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

In [0]:
# Load the CIFAR100 data.
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    #print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=100):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=100):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [0]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
#model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar100_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler] # [checkpoint] # 

In [0]:
def get_losses(x_in, y_in, model):
  loss_arr = []
  for i in range(len(x_in)):
    loss_arr.append(model.evaluate(x_in[i:i+1],
                                  y_in[i:i+1],
                                batch_size=None,
                                verbose=0,
                                steps=1)[0])
  return loss_arr

In [0]:
def train_with_growing_sets(X_train, Y_train, X_test, Y_test, model, sets_method=None, 
                            data_limit=None, div=16, num_epochs=30, batch_size=128):
  """
  sets_method:  (String) Name of the growing sets method you want the train with. e.g. "SPL","SPLI","ROGS","BASE"
  data_limit: (Int) If you dont want to feed the model with all of your data, you can limit it.
  div:  (Int) How many pieces you want to split your data. Those pieces will add up together in every step.
  num_epochs:  (Int) Epoch for each training set.
  """
  custom_methods = ["spl", "spli", "rogs", "base"]

  is_str = isinstance(sets_method, str)
  if is_str:
    sets_method = sets_method.lower()
  if not is_str  or sets_method not in custom_methods:
    print("Sets method '{}' cannot recognized.".format(sets_method))
    return
    
  if not data_limit:
    data_limit = len(X_train)

  X_train, Y_train = X_train[:data_limit], Y_train[:data_limit]

  print("Method: {} - Div: {} - Epoch: {}".format(sets_method.upper(),k_div,num_epochs))
  start_t = datetime.datetime.now().replace(microsecond=0)

  slen = int(len(X_train)/k_div)
  spl_accs,rspl_accs,rogs_accs,accs  = [],[],[],[]
  sizes = []

  if sets_method == "spl":
    # Self Paced Learning
    for i in range(k_div):
      losses = get_losses(X_train, Y_train, model)
      sorted_loss_indexes = np.argsort(losses)
      
      x_tmp_spc = []
      y_tmp_spc = []
      for i in range(i * slen + slen):
        x_tmp_spc.append(X_train[sorted_loss_indexes[i]])
        y_tmp_spc.append(Y_train[sorted_loss_indexes[i]])

      x_spc = np.asarray(x_tmp_spc)
      y_spc = np.asarray(y_tmp_spc)

      hist = model.fit(x_spc, y_spc, epochs=num_epochs, shuffle=False, batch_size=batch_size, verbose=0)
      print("Instant LEN: {}".format(len(x_spc)))

      inst_acc = model.evaluate(X_test, Y_test)[1]
      print("Acc: {}".format(inst_acc))
      spl_accs.append(inst_acc)
      sizes.append(len(x_spc))
    print("{}_accs = {}".format(sets_method, spl_accs))

  elif sets_method == "spli":
    # Reversed Self Paced Learning
    for i in range(k_div):
      losses = get_losses(X_train, Y_train, model)
      sorted_loss_indexes = np.argsort(losses)[::-1]
      
      x_tmp_spc = []
      y_tmp_spc = []
      for i in range(i * slen + slen):
        x_tmp_spc.append(X_train[sorted_loss_indexes[i]])
        y_tmp_spc.append(Y_train[sorted_loss_indexes[i]])

      x_spc = np.asarray(x_tmp_spc)
      y_spc = np.asarray(y_tmp_spc)

      hist = model.fit(x_spc, y_spc, epochs=num_epochs, shuffle=False, batch_size=batch_size, verbose=0)
      print("Instant LEN: {}".format(len(x_spc)))

      inst_acc = model.evaluate(X_test, Y_test)[1]
      print("Acc: {}".format(inst_acc))
      rspl_accs.append(inst_acc)
      sizes.append(len(x_spc))
    print("{}_accs = {}".format(sets_method, rspl_accs))

  elif sets_method == "rogs":
    # Random Ordered Growing Datasets
    for i in range(k_div):
      x_part = X_train[: i * slen + slen]
      y_part = Y_train[: i * slen + slen]
      hist = model.fit(x_part, y_part, epochs=num_epochs, shuffle=False, batch_size=batch_size, verbose=0)
      print("Instant LEN: {}".format(len(x_part)))

      inst_acc = model.evaluate(X_test, Y_test)[1]
      print("Acc: {}".format(inst_acc))
      rogs_accs.append(inst_acc)
      sizes.append(len(x_part))
    print("{}_accs = {}".format(sets_method, rogs_accs))

  elif sets_method == "base":
    # Baseline training
    for _ in range(num_epochs):
      hist = model.fit(X_train, Y_train, epochs=1, shuffle=True, batch_size=batch_size, verbose=0) 

      inst_acc = model.evaluate(X_test, Y_test)[1]
      print("Acc: {}".format(inst_acc))
      accs.append(inst_acc)
      sizes.append(len(X_train))
    print("{}_accs = {}".format(sets_method, accs))

  end_t = datetime.datetime.now().replace(microsecond=0)
  print("{}_sizes = {}".format(sets_method, sizes))
  print("Time elapsed: {}".format(end_t - start_t))

In [0]:
num_epochs = 10
k_div=10
batch_size = 400

train_with_growing_sets(x_train, y_train, x_test, y_test, model, sets_method="base",
                            data_limit=100, div=k_div, num_epochs=num_epochs, batch_size=batch_size)

In [0]:
# LOAD MODEL
name = "empty_model"
modelpath = "/.../" + name

# Model reconstruction from JSON file
with open(modelpath + ".json", 'r') as f:
    model = model_from_json(f.read())
# Load weights into the new model
model.load_weights(modelpath + ".h5")
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=lr_schedule(0)), #@@@@@
              metrics=['accuracy'])

In [0]:
#SAVE MODEL
name = "model_name1"
modelpath = "/<data_directory>/" + name
print(modelpath)
model.save_weights(modelpath + ".h5")
with open(modelpath + ".json", 'w') as f:
    f.write(model.to_json())